## Analyze whether SNWD varies more from year to year or from place to place.

In [111]:
import pandas as pd
import numpy as np
import urllib
import math

In [112]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [113]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [114]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSSSBB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1271
+-------------------+------------------+-------------------+------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+-------------------+------------------+-----------+-----------+------+--------------------+------+
|            coeff_1|           coeff_2|            coeff_3|           coeff_4|elevation|   label|latitude|longitude|measurement|             res_1|              res_2|              res_3|              res_4|          res_mean|    station|  total_var|undefs|              vector|  year|
+-------------------+------------------+-------------------+------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+-------------------+------------------+-----------+-----------+------+--------------------+------+
|-1956.2140564896897| 2409.747966697589| -340.2037727766924|-856

In [115]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [116]:
#extract longitude and latitude for each station
feature='coeff_4'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_4'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_4 FROM weather


station    year      coeff_4
0  USC00027661  1969.0  -856.353236
1  USC00028499  1955.0  -775.388626
2  USC00025467  1935.0  -706.982372
3  USC00028499  1979.0 -1057.479092
4  USC00021026  1953.0  -972.788797

In [117]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_4')
year_station_table.head(10)

station  USC00020060  USC00020104  USC00020406  USC00020949  USC00021026  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN  -783.557861   
1902.0           NaN          NaN          NaN          NaN  -656.845841   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station  USC00021161  USC00021282  USC00021353  USC00021356  USC00021361  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00029634  USC00040924  USC00042598  USC00043855  \
year        ...                                                            
1901.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1903.0      ...               NaN          NaN          NaN          NaN   
1904.0      ...               NaN          NaN          NaN          NaN   
1905.0      ...               NaN          NaN          NaN          NaN   
1906.0      ...               NaN          NaN          NaN          NaN   
1907.0      ...               NaN          NaN          NaN          NaN   
1908.0      ...               NaN          NaN          NaN          NaN   
1909.0      ...               NaN          NaN          NaN          NaN   
1910.0      ...               NaN          NaN          NaN          NaN   

station  USC00044259  USC00045502  USC00045860  USC00046635  USW00023183  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0    -370.67521          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station  USW00093140  
year                  
1901.0           NaN  
1902.0           NaN  
1903.0           NaN  
1904.0           NaN  
1905.0           NaN  
1906.0           NaN  
1907.0           NaN  
1908.0           NaN  
1909.0           NaN  
1910.0           NaN  

[10 rows x 47 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [118]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'fraction ',100*(1-1.0*RMS(tbl_minus_station)/RMS(year_station_table))
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)
print 'fraction ',100*(1-1.0*RMS(tbl_minus_year)/RMS(year_station_table))

total RMS                   =  832.562054782
RMS removing mean-by-station=  171.232805167
fraction  79.4330279426
RMS removing mean-by-year   =  129.581797357
fraction  84.4357790975


In [119]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 832.562054782
0 after removing mean by year    = 129.581797357
0 after removing mean by stations= 119.451855408
1 after removing mean by year    = 118.653834684
1 after removing mean by stations= 118.5289497
2 after removing mean by year    = 118.495841305
2 after removing mean by stations= 118.48404252
3 after removing mean by year    = 118.479229736
3 after removing mean by stations= 118.477121418
4 after removing mean by year    = 118.47615492
4 after removing mean by stations= 118.475697848


In [120]:
sc.stop()